In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
from skimage import io, transform
import PIL
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./image_files', train=True,
                                        download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [3]:
def imshow(img):
    unnormalized_image = img / 2 + 0.5     # unnormalize
    npimg = unnormalized_image.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.conv3 = nn.Conv2d(32, 32, 3)
        self.fc1 = nn.Linear(32 * 2 * 2, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 32 * 2 * 2)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [5]:
net = Net().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)

In [6]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs_cpu, labels_cpu = data
        inputs, labels = inputs_cpu.cuda(), labels_cpu.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0


[1,   100] loss: 2.303
[1,   200] loss: 2.290
[1,   300] loss: 2.184
[1,   400] loss: 2.050
[1,   500] loss: 1.975
[1,   600] loss: 1.939
[1,   700] loss: 1.839
[1,   800] loss: 1.811
[1,   900] loss: 1.744
[1,  1000] loss: 1.714
[1,  1100] loss: 1.673
[1,  1200] loss: 1.682
[1,  1300] loss: 1.592
[1,  1400] loss: 1.633
[1,  1500] loss: 1.575
[1,  1600] loss: 1.518
[1,  1700] loss: 1.482
[1,  1800] loss: 1.503
[1,  1900] loss: 1.520
[1,  2000] loss: 1.446
[1,  2100] loss: 1.476
[1,  2200] loss: 1.483
[1,  2300] loss: 1.448
[1,  2400] loss: 1.448
[1,  2500] loss: 1.390
[1,  2600] loss: 1.407
[1,  2700] loss: 1.402
[1,  2800] loss: 1.401
[1,  2900] loss: 1.377
[1,  3000] loss: 1.372
[1,  3100] loss: 1.364
[2,   100] loss: 1.365
[2,   200] loss: 1.318
[2,   300] loss: 1.308
[2,   400] loss: 1.283
[2,   500] loss: 1.293
[2,   600] loss: 1.334
[2,   700] loss: 1.280
[2,   800] loss: 1.281
[2,   900] loss: 1.210
[2,  1000] loss: 1.243
[2,  1100] loss: 1.273
[2,  1200] loss: 1.185
[2,  1300] 

In [7]:
testset = torchvision.datasets.CIFAR10(root='./image_files', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)


In [8]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images_cpu, labels_cpu = data
        images, labels = images_cpu.cuda(), labels_cpu.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 66 %


In [9]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images_cpu, labels_cpu = data
        images, labels = images_cpu.cuda(), labels_cpu.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


Accuracy of plane : 68 %
Accuracy of   car : 82 %
Accuracy of  bird : 63 %
Accuracy of   cat : 44 %
Accuracy of  deer : 40 %
Accuracy of   dog : 54 %
Accuracy of  frog : 69 %
Accuracy of horse : 64 %
Accuracy of  ship : 84 %
Accuracy of truck : 66 %
